In [153]:
from config import API_KEY, API_SECRET_KEY, BEARER_TOKEN, ACCESS_TOKEN, ACCESS_TOKEN_SECRET, EXCEPTIONS, SCREEN_NAME
import tweepy
import time

In [167]:
# Requests / 3-hour window	300* per user; 300* per app
# https://developer.twitter.com/en/docs/twitter-api/v1/tweets/post-and-engage/api-reference/post-statuses-update
# https://developer.twitter.com/en/docs/twitter-api/tweets/filtered-stream/api-reference/get-tweets-search-stream-rules

In [154]:
def connect():
    """Setting up the api connection"""
    auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)  # Read&Write Permissions
    return tweepy.API(auth)

In [155]:
def connection_verified(api):
    """Verifying twitter api connection"""
    print('Verifying connection..')
    try:
        api.verify_credentials()
        print('Verified')
        return True
    except:
        print('Not Verified...')
        return False

In [ ]:
def main():
    api = connect()
    if not connection_verified(api):
        return False
    return None

In [166]:
# ____ = main()